In [4]:
!python3 -m pipenv --version

pipenv, version 2022.10.9


In [7]:
!curl https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin --output model1.bin
!curl https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin --output dv.bin

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   889  100   889    0     0  11160      0 --:--:-- --:--:-- --:--:-- 12013
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   333  100   333    0     0   2804      0 --:--:-- --:--:-- --:--:--  3027


In [2]:
# Write a script for loading these models with pickle
import pickle
import json
import numpy as np
import pandas as pd

with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)
    
with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [17]:
# Score this client:

client_data = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

# Write a function that takes a client_data and dv object and returns a prediction
def predict(client_data, dv, model):
    X = dv.transform([client_data])
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0]

churn = predict(client_data, dv, model)
print(churn)

0.16213414434326598


In [14]:
# Write Flask code for serving the model
from werkzeug.wrappers import Request, Response
from flask import Flask, jsonify

app = Flask('churn')

@app.route('/predict', methods=['POST'])
def predict():
    customer = Request.get_json()
    churn = predict(customer, dv, model)
    result = {
        'churn_probability': float(churn),
        'churn': bool(churn > 0.5)
    }
    return jsonify(result)

# Run the server
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)


 * Running on http://localhost:9000 (Press CTRL+C to quit)
127.0.0.1 - - [09/Oct/2022 18:29:56] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Oct/2022 18:29:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Oct/2022 18:30:11] "GET /predict HTTP/1.1" 405 -
